In [469]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import norm
from scipy.special import boxcox1p
import missingno as msno
from ast import literal_eval


import warnings
warnings.filterwarnings('ignore')

pd.set_option("display.float_format", lambda x: "{:.3f}".format(x)) #Limiting floats output to 3 decimal points
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
#pd.set_option("display.max_colwidth", None)

In [470]:
df = pd.read_csv("./files/Sligo_residential_listings.csv")

In [471]:
df.shape

(364, 30)

In [472]:
df.duplicated(subset='id').any()

False

In [473]:
missing_percent = df.isnull().mean()
missing_percent.sort_values(ascending=False)

openViewings       0.995
priceHistory       0.984
sticker            0.956
prs                0.805
image              0.805
propertySize       0.728
floorArea          0.728
numBathrooms       0.646
newHome            0.522
propertyType       0.475
pageBranding       0.261
numBedrooms        0.173
ber                0.165
saleType           0.000
featuredLevel      0.000
abbreviatedPrice   0.000
publishDate        0.000
price              0.000
seoTitle           0.000
title              0.000
sections           0.000
media              0.000
daftShortcode      0.000
seller             0.000
point              0.000
seoFriendlyPath    0.000
category           0.000
state              0.000
premierPartner     0.000
id                 0.000
dtype: float64

### Drop columns have more 80% missing values. 
* ['sticker', 'prs', 'image', 'priceHistory', 'offers', 'openViewings', 'newHome']
    * sticker: won't be an important factor for the house price prediction.
    * openViewings won't be important factors for the house price prediction.
    * priceHistory: this might be useful if we want to compare with the current price, but most listings do not have this data. Therefore drop this column too.
    * image: The image quality is important for a house website, but we will use other columns to determine house quality instead of images.
   * prs: the column gathered same info in the other columns, not useful to have this column. 
        * df[~df.prs.isnull()]["prs"]  <--- checked with this code
    * newHome: can be found in other column.



In [474]:
inds = df.isnull().mean()>0.8
drop_cols = list(inds[inds==True].index)
df.drop(drop_cols, axis=1, inplace=True)

### Drop duplicated or irrelevant columns to our price predictions

In [475]:
#title and seoTile are identical, drop seoTitle
df.drop("seoTitle", axis=1, inplace=True)

#saleType only has a single value "For Sale", not useful.
df.saleType.unique()
df.drop("saleType", axis=1, inplace=True)

#Price and abbreviatedPrice are same thing in different formats, drop abbreviatedPrice.
df.drop("abbreviatedPrice", axis=1, inplace=True)

# A unique code for daft website, not useful for our price prediction.
df.drop("daftShortcode", axis=1, inplace=True)

# A link path to the property page, not useful for the price prediction.
df.drop("seoFriendlyPath", axis=1, inplace=True)

# Contains logo info, not useful for the price prediction.
df.drop("pageBranding", axis=1, inplace=True)

# Only contains one value "Published", not useful.
df.drop("state", axis=1, inplace=True)

# Only contains one value False, not useful.
df.drop("premierPartner", axis=1, inplace=True)

# Features of new homes, homes are not new won't have these features
df.drop("newHome", axis=1, inplace=True)


### Cleaning data

#### Parse columns with string dictionary: point

In [476]:
print(df.shape)

# Convert dictionary inside the "point" column to multiple columns
df.point = df.point.apply(literal_eval)
df = df.join(pd.json_normalize(df.point))
df.drop(["point", "type"], axis=1, inplace=True)

print(df.coordinates.isnull().any())

# split coordinate to longitude and latitude
df = df.join(pd.DataFrame(df.pop('coordinates').tolist(), index=df.index, columns=["longitude", "latitude"]))

print(df.shape)

(364, 16)
False
(364, 17)


#### Parse columns with string dictionary: seller

In [477]:
# Convert the column of stringified dicts to dicts
df.seller = df.seller.map(literal_eval)
# normalize the column of dictionaries and join it to df
df = df.join(pd.json_normalize(df.seller))
df.drop(columns=['seller'], inplace=True)

# rename ber related column names for the clarity
df.rename(columns={"name":"seller_name", "branch":"seller_branch"}, inplace=True)
df.drop(['phone', 'profileImage', 'address', 'standardLogo', 'squareLogo', 'backgroundColour', 'licenceNumber',
       'showContactForm', 'alternativePhone', 'phoneWhenToCall'], axis=1, inplace=True)

# seller_branch has NA values because the property is dold my private users.
df.loc[df.sellerType == 'PRIVATE_USER', "seller_branch"] = "private"

# change data type
df.sellerId = df.sellerId.astype(str)

print(df.shape)

(364, 20)


#### Parse columns with string dictionary: media

In [478]:
# Convert the column of stringified dicts to dicts
df.media = df.media.apply(literal_eval)

# normalize the column of dictionaries and join it to df
df = df.join(pd.json_normalize(df.media))

# drop columns from media that we do not need
df.drop(["media"], axis=1, inplace=True)
df.drop(["images"], axis=1, inplace=True)
df.drop(["brochure"], axis=1, inplace=True)

# rename media related columns names for the clarity
df.rename(columns={"totalImages":"m_totalImages", "hasVideo":"m_hasVideo", "hasVirtualTour":"m_hasVirtualTour", "hasBrochure":"m_hasBrochure", }, inplace=True)

# change data types

df.m_hasVideo = df.m_hasVideo.astype(bool)
df.m_hasBrochure = df.m_hasBrochure.astype(bool)
df.m_hasVirtualTour = df.m_hasVirtualTour.astype(bool)
df.m_totalImages = df.m_totalImages.astype("float")
print(df.shape)

(364, 23)


#### Parse columns with string dictionary: ber (Building Energy Rating)

In [479]:
# parse the 'ber'(Building Energy Rating)column
# More details about BER: https://www.churches.ie/faq-what-do-the-different-ber-ratings-mean/
#df = df[df['ber'].notna()]
# replace NaN with '{}' if the column is strings, otherwise replace with {}
df.ber = df.ber.str.replace("NA", "XXX")
df.ber = df.ber.fillna('{"rating": "XXX"}')  # if the NaN is in a column of strings
#df.ber = df.ber.fillna({i: {} for i in df.index})  # if the column is not strings

df.ber = df.ber.apply(literal_eval)
df = df.join(pd.json_normalize(df.ber))
df.drop(columns=['ber'], inplace=True)

# rename ber related column names for the clarity
# epi stands for Energy Performance Index
df.rename(columns={"rating":"ber_rating", "epi":"ber_epi"}, inplace=True)
df.drop(["code"], axis=1, inplace=True)

# epi is missing half of the data. We will use ber_rating instead to measure the Energy Rating for the building.
df.drop(["ber_epi"], axis=1, inplace=True)

print(df.shape)

(364, 23)


#### Parse columns with string dictionary: floorArea

In [480]:
# parse the 'floorArea' column
# replace NaN with '{}' if the column is strings, otherwise replace with {}
df.floorArea = df.floorArea.fillna('{"unit": "METRES_SQUARED", "value": "999"}')  # if the NaN is in a column of strings
#df.floorArea = df.floorArea.fillna({i: {} for i in df.index})  # if the column is not strings

# convert string of dict to dict
df.floorArea = df.floorArea.apply(literal_eval)
# parse dict and join it to df
df = df.join(pd.json_normalize(df.floorArea))
# drop floorArea
df.drop(["floorArea"], axis=1, inplace=True)

# rename ber related column names for the clarity
df.rename(columns={"unit":"floorArea_unit", "value":"floorArea_size"}, inplace=True)

# convert dtype
df.floorArea_size = df.floorArea_size.astype("float")


print(df.shape)

(364, 24)


In [481]:
# convert featureLevel from all caps to lower case just for the convinience
df.featuredLevel = df.featuredLevel.str.lower()

###############PublishDate#################
# convert publishDate from int64 to datetime format
df["publishDate"] = pd.to_datetime(df["publishDate"].astype("str"), utc=True, unit="ms").dt.strftime("%Y-%m-%d")


###############Price#################
# remove rows do not have price or contain pound sign. 
# four rows contained pound sign, which indicate it is not a property in Ireland.
df = df.drop(df[df.price.str.contains("Price on Application") == True].index)
print(df.shape)

# replace , € [] to blank
df["price"] = df.price.str.replace(',','').str.strip()
df["price"] = df.price.str.replace('€','').str.strip()
df["price"] = df.price.str.replace('(','').str.strip()
df["price"] = df.price.str.replace(')','').str.strip()
df["price"] = df.price.str.replace('[','').str.strip()
df["price"] = df.price.str.replace(']','').str.strip()
df["price"] = df.price.str.replace('From ','').str.strip()

#AMV stands for Advides Minimum Values. Add AMV_price flag in case it is useful.
# After marking AMV, we no longer need AMV letter in the price column, replace AMV with blank.
df['AMV_price'] = 0
df.loc[df[df.price.str.startswith('AMV') == True].index, 'AMV_price'] = 1
df["price"] = df.price.str.replace('AMV: ','').str.strip()

df["price"] = df.price.astype(int)


###############Bedrooms&Bathrooms#################
# drop rows with multiple property types
temp = df[df.numBedrooms.str.contains("&") == True]
df.drop(temp.index, inplace=True)
print(df.shape)

# remove letters from the columns
df.numBedrooms = df.numBedrooms.str.replace(r"[^0-9]", '', regex=True).str.strip()
df.numBathrooms = df.numBathrooms.str.replace(r"[^0-9]", '', regex=True).str.strip()


# filling nan values with one for columns numBedroos and numBathrooms
df.numBedrooms = df.numBedrooms.fillna(1)
df.numBathrooms = df.numBathrooms.fillna(1)

# chage dtypes
df.numBedrooms = df.numBedrooms.astype("int")
df.numBathrooms = df.numBathrooms.astype("int")


###############PropertyType#################
# filling nan values in the propertyType column
print(df[df.propertyType.isnull()]["sections"].unique())

# fill propertyType based on the "category" column
df.propertyType = df.propertyType.fillna("Houses")

# sections can be dropped as the same info presented in properyType.
df.drop("sections", axis=1, inplace=True)

(228, 24)
(188, 25)
["['Property', 'New Homes', 'Houses']"]


In [482]:
print(df.shape)

(188, 24)


#### Unit conversion in propertySize

In [483]:
# mark rows with "ac" in propertySize, ac stands for Acre
df["ac_flag"] = df.propertySize.str.contains("ac")
# remove unit strings from propertySize and conver the column to float type
df['propertySize'] = df['propertySize'].replace({'m²': '', 'ac': ''}, regex=True).astype(float)

print(df.shape)
# Acres to square meters are 4046.86, but the converted values are way bigger than the propertySize from the daft.ie websites.
# The website also shows acres and square meters, but the conversion rate is not 4046.86.
# As the units and the conversion rates are not consistent in the website, drop rows with acres as unit.
inds = df[(df.floorArea_unit=="ACRES")|df.ac_flag==True].index
df.drop(inds, inplace=True)

# substitute flooarArea_size for propertySize NAN values
df.loc[(df.propertySize.isnull()) & (df.floorArea_size!=999), ["propertySize"]] = df.floorArea_size

print(df.shape)

(188, 25)
(140, 25)


In [484]:
df.drop(["floorArea_unit", "floorArea_size", "ac_flag"], axis=1, inplace=True)

### Export to csv

In [485]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140 entries, 0 to 361
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                140 non-null    int64  
 1   title             140 non-null    object 
 2   featuredLevel     140 non-null    object 
 3   publishDate       140 non-null    object 
 4   price             140 non-null    int64  
 5   numBedrooms       140 non-null    int64  
 6   numBathrooms      140 non-null    int64  
 7   propertyType      140 non-null    object 
 8   category          140 non-null    object 
 9   propertySize      40 non-null     float64
 10  longitude         140 non-null    float64
 11  latitude          140 non-null    float64
 12  sellerId          140 non-null    object 
 13  seller_name       140 non-null    object 
 14  seller_branch     140 non-null    object 
 15  sellerType        140 non-null    object 
 16  m_totalImages     140 non-null    float64
 1

In [486]:
df.shape

(140, 22)

In [487]:
df.isnull().sum()

id                    0
title                 0
featuredLevel         0
publishDate           0
price                 0
numBedrooms           0
numBathrooms          0
propertyType          0
category              0
propertySize        100
longitude             0
latitude              0
sellerId              0
seller_name           0
seller_branch         0
sellerType            0
m_totalImages         0
m_hasVideo            0
m_hasVirtualTour      0
m_hasBrochure         0
ber_rating            0
AMV_price             0
dtype: int64

In [488]:
df.tail()

,id,title,featuredLevel,publishDate,price,numBedrooms,numBathrooms,propertyType,category,propertySize,longitude,latitude,sellerId,seller_name,seller_branch,sellerType,m_totalImages,m_hasVideo,m_hasVirtualTour,m_hasBrochure,ber_rating,AMV_price
354,1117985,"Martello, Martello, Rushbrooke, Cobh, Co. Cork",basic,2021-09-28,345000,4,1,Houses,New Homes,nan,-8.316,51.850,2559,Paul O'Shea,Sherry FitzGerald Cork,BRANDED_AGENT,34.000,False,False,False,A3,0
357,1485530,"Clog Na Léinn, Collinstown, Co. Westmeath",basic,2022-02-07,395000,4,1,Houses,New Homes,nan,-7.212,53.647,366,Robert Forbes,Forbes Property,BRANDED_AGENT,24.000,False,True,False,A1,0
358,1117980,"Castle Park, Killycard, Castleblayney, Co. Mon...",basic,2022-01-14,269000,3,1,Houses,New Homes,nan,-6.754,54.115,4705,DNG John O' Brien Office,DNG John O’Brien,UNBRANDED_AGENT,14.000,True,False,False,A3,0
359,1118108,"SeaGreen Park, Sea, Green Park, Blacklion, Gre...",basic,2022-01-31,545000,4,1,Houses,New Homes,nan,-6.086,53.146,9634,Jackie Horan,Sherry FitzGerald New Homes,BRANDED_AGENT,25.000,False,False,False,A2,0
361,1137830,"Revington Gardens, North Circular Road, North ...",basic,2021-06-21,650000,4,1,Houses,New Homes,nan,-8.653,52.665,2299,Tom Hickey,Hickey O'Donoghue Auctioneers Ltd.,BRANDED_AGENT,38.000,False,True,False,A2,0


In [489]:
df.to_csv("./output/daft_sligo_v1.csv", index=False)